In [1]:
# !wget -O quora.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1ERtxpdWOgGQ3HOigqAMHTJjmOE_tWvoF"
# !unzip quora.zip
# !pip install -q --upgrade nltk gensim bokeh pandas

# import numpy as np
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# np.random.seed(42)

In [2]:
# !pip install -qq nltk==3.4
#!conda install -c anaconda -y gensim
# !pip install -qq pandas==0.23.4
# !pip install -qq bokeh==1.0.3

# !wget -O quora.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1ERtxpdWOgGQ3HOigqAMHTJjmOE_tWvoF"
# !unzip quora.zip

In [4]:
import numpy as np
import nltk

np.random.seed(42)

# nltk.download('punkt')
# nltk.download('stopwords')

# Word Embeddings

*NB. This notebook is somewhat based on the YSDA NLP course [notebook](https://github.com/yandexdataschool/nlp_course/tree/master/week01_embeddings).*

Guess, you've seen such pictures already:  
![embeddings relations](https://www.tensorflow.org/images/linear-relationships.png)
*From [Vector Representations of Words, Tensorflow tutorial](https://www.tensorflow.org/tutorials/representation/word2vec)*

We are going to use these thingies alo-o-ot.

Well, we need a proper introduction, nevertheless. Do you remember how we represented sentences last time?

We converted a sentence to the bag-of-words:  
![](https://i.ibb.co/Tvw1c8S/BOW.png)

And each word was represented using one-hot encoding (a vector with one at the position corresponding to the word's index and zeros at all others positions).

These one-hot encoding vectors have extremely high dimensions (like, hundreds of thousands or millions). They fit their purpose - to encode information about words. But they have several disadvantages.

First of all, they are almost uninterpretable. I mean, all one-hot encoding vectors are orthonormal, so you cannot say that, e.g. `man` and `men` are more similar words than `man` and `crocodiles`.

But we want to. Well, NLP researchers in the past few years wanted to, cannot really speak for you.

And we're gonna build vectors, that encode semantics!

Look at the first picture. It shows relations encoded in the word embeddings space. Such as male-female or verb tense... whatever, just check these two links: http://bionlp-www.utu.fi/wv_demo/, https://lamyiowce.github.io/word2viz/. Go and play with this relations right now! They are funny and you'll get an insight into what the word embeddings can.

There is another disadvantage of one-hot encoding vectors: their size. The word embedding vectors we are going to play with have dimensions from 50 to 600 usually. That is by a few orders of magnitude smaller than one-hot encoding vectors.

This is crucial for neural networks - they can work only with sufficiently small dense vectors. Well, we'll speak about it later.

---

In this notebook, we are going to work with [gensim](https://radimrehurek.com/gensim/) - somewhat standard word embeddings python library. We'll just superficially discuss how it works, but we'll train our model and apply a pretrained one. As a result, you're (probably) gonna understand how to work with word embeddings.

In the next notebook, we'll try to work out how word embeddings work and how to implement a module to train word embeddings.

## Training Model

Well, nothing is interesting in mere training of the word embeddings model. We are gonna apply it to a very concrete task: [Quora Question Pairs](https://www.kaggle.com/c/quora-question-pairs) from kaggle:

In [5]:
import pandas as pd

quora_data = pd.read_csv('train.csv')

# quora_data.sample(20)[['question1', 'question2', 'is_duplicate']]

In [6]:
quora_data.sample(20)[['question1', 'question2', 'is_duplicate']]

,question1,question2,is_duplicate
8067,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,0
368101,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...,0
70497,Which is more advisable and better material fo...,What is the best server setup for buddypress?,0
226567,How do I improve logical programming skills?,How can I improve my logical skills for progra...,1
73186,How close we are to see 3rd world war?,How close is a World War III?,1
215105,What do Chinese people think about Donald Trump?,What do Chinese people think of Donald Trump?,1
253209,How many hours a week do Google employees work?,How many hours a day do Google employees work ...,0
354651,How can we follow a Quora question privately w...,How can we view private Instagram pictures wit...,0
104478,Why are cats so overprotective?,How do you know if your cat is overprotective?,1
163628,How do I improve logical programming skills?,What is the best way to improve logical skills...,1


In [7]:
#!pip install --upgrade pandas

In [8]:
#quora_data.sample(20)[['question1', 'question2', 'is_duplicate']]

You see, the dataset consists of question pairs and you have to determine which of them are duplicates and which are not.

Well, I'm not promising that we'll achieve good results right now, but still... Let's train Word2vec gensim model!

*Word2vec is the most popular method of building word embeddings. We'll implement it next time, right now let's believe that it just do whatever we want.*

First of all, we need to collect available texts to pass them to Word2vec model:

In [9]:
import numpy as np

quora_data.question1 = quora_data.question1.replace(np.nan, '', regex=True)
quora_data.question2 = quora_data.question2.replace(np.nan, '', regex=True)

texts = list(pd.concat([quora_data.question1, quora_data.question2]).unique())
texts[:10]

['What is the step by step guide to invest in share market in india?',
 'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
 'How can I increase the speed of my internet connection while using a VPN?',
 'Why am I mentally very lonely? How can I solve it?',
 'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?',
 'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?',
 'Should I buy tiago?',
 'How can I be a good geologist?',
 'When do you use シ instead of し?',
 'Motorola (company): Can I hack my Charter Motorolla DCX3400?']

Next, we have to tokenize the texts. This time we'll use `nltk` - another great NLP library. 

It goes this way:

In [10]:
from nltk.tokenize import word_tokenize

word_tokenize(texts[0])

['What',
 'is',
 'the',
 'step',
 'by',
 'step',
 'guide',
 'to',
 'invest',
 'in',
 'share',
 'market',
 'in',
 'india',
 '?']

**Task** Your turn: lowercase all the texts and tokenize them:

In [11]:
tokenized_texts = [*map(lambda text: word_tokenize(text.lower()), texts)]

assert all(isinstance(row, (list, tuple)) for row in tokenized_texts), \
    "please convert each line into a list of tokens"
assert all(all(isinstance(tok, str) for tok in row) for row in tokenized_texts), \
    "please convert each line into a list of tokens"

is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(not is_latin(token) or token.islower() for tokens in tokenized_texts for token in tokens),\
    "please lowercase each line"

In [12]:
print([' '.join(row) for row in tokenized_texts[:2]])

['what is the step by step guide to invest in share market in india ?', 'what is the story of kohinoor ( koh-i-noor ) diamond ?']


And we are ready to train a small model:

In [13]:
from gensim.models import Word2Vec

model = Word2Vec(tokenized_texts, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5,     # define context as a 5-word window around the target word
                 seed=0,       # + workers=1 is to make model reproducible
                 workers=1).wv

C:\Users\agama\anaconda3\envs\course\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Analyzing Model

Yay, we have our own model, let's play with it!

To get word's vector, well, call `get_vector`:

In [14]:
model.get_vector('anything')

array([-0.6059568 ,  2.460762  , -0.5609757 , -0.6594619 , -3.2099524 ,
        2.4934635 ,  0.9850607 ,  0.5102441 , -1.0140994 ,  2.0118055 ,
       -1.815692  , -4.2941537 , -1.9720241 , -0.02941428,  3.9537377 ,
        0.21149045, -3.5083218 ,  1.6357796 ,  2.3724627 ,  2.1486197 ,
        3.8126264 , -3.0769866 , -1.653677  , -0.72145635, -1.6492962 ,
       -0.757187  ,  0.23071285,  0.01671845,  1.6631601 ,  0.03714013,
        2.1424541 ,  1.8517803 ], dtype=float32)

To get most similar words for the given one (guess, what):

In [15]:
model.most_similar('bread')

[('rice', 0.9563511610031128),
 ('fruit', 0.947577178478241),
 ('butter', 0.9317622184753418),
 ('vodka', 0.9248947501182556),
 ('sauce', 0.9244229197502136),
 ('beans', 0.914040207862854),
 ('honey', 0.9098433256149292),
 ('banana', 0.9076601266860962),
 ('flour', 0.906358003616333),
 ('pasta', 0.9051861763000488)]

And it can do such magic:

In [16]:
model.most_similar(positive=['coder', 'money'], negative=['brain'])

[('photographer', 0.6976268291473389),
 ('betting', 0.6549664735794067),
 ('volunteer', 0.6211757063865662),
 ('writer', 0.6180971264839172),
 ('youtuber', 0.6097328662872314),
 ('athlete', 0.597032368183136),
 ('gifts', 0.5961406230926514),
 ('conference', 0.5952812433242798),
 ('basketball', 0.586754560470581),
 ('tutor', 0.5855691432952881)]

That is, who is like coder, with money and without brains.

And this too:

In [17]:
model.most_similar([model.get_vector('politician') - model.get_vector('power') + model.get_vector('honesty')])

[('romantic', 0.7079866528511047),
 ('actress', 0.6925302147865295),
 ('girl/woman', 0.6619561910629272),
 ('ted', 0.6391326189041138),
 ('bhatt', 0.6329595446586609),
 ('alia', 0.615281343460083),
 ('thoughtful', 0.6080158352851868),
 ('effortlessly', 0.6065546274185181),
 ('astrologer', 0.6039962768554688),
 ('farewell', 0.6033608317375183)]

Honest politician without power, isn't it just cute.

**Task** Play with it. And yes, I'm serious.

In [18]:
model.most_similar(positive=['doctor','woman'], negative=['man'])

[('teacher', 0.7798760533332825),
 ('therapist', 0.7773643732070923),
 ('married', 0.7656753063201904),
 ('teenager', 0.7491468787193298),
 ('girlfriend', 0.7469215393066406),
 ('dentist', 0.7425242066383362),
 ('girl', 0.7387226223945618),
 ('student', 0.7370373010635376),
 ('professor', 0.7302892208099365),
 ('lawyer', 0.7270255088806152)]

## Visualizing Model

Let's now look at the projection of the first 1000 the most frequent words.

In [19]:
words = sorted(list(model.key_to_index), #model.vocab.keys()
               key=lambda word: model.get_vecattr(word, "count"), #model.vocab[word].count
               reverse=True)[:1000]

print(words[::100])

['?', 'money', 'study', 's', '6', 'physics', 'says', 'view', 'meet', 'changed']


**Task** Build the matrix from these words' vectors.

In [20]:
word_vectors = model.vectors[[model.key_to_index[word] for word in words]]

assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), model.vectors.shape[1])
assert np.isfinite(word_vectors).all()

Now we would try to project this 32 dimensional vectors to the more convenient 2D space to be able to look on them.

### PCA

The simplest linear method of dimension reduction is __P__rincipial __C__omponent __A__nalysis.

PCA builds so called principal components - set of variables along which our data has the largest variance:  

![pca](https://i.stack.imgur.com/Q7HIP.gif)
*From the great answer [https://stats.stackexchange.com/a/140579](https://stats.stackexchange.com/a/140579)*

For instance, in the picture, the rotating line represents possible variants of the first principal component. If we want to project 2D set of dots to one dimension, we'll probably want to save as much information as possible. The maximum variance position of the rotating line gives us more information about the dots than all other positions.

Really nice illustrations of this mechanism live [here](http://setosa.io/ev/principal-component-analysis/).

To be short, project multi-dimensional space on the first two or three components and enjoy fast-and-dirty dimensional reduction.

**Task** Use [sklearn PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to project data to 2D. Centre and normalize the output.

In [21]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def get_pca_projection(word_vectors):
    # <implement me>
    pca = PCA(n_components=2)
    word_vectors = pca.fit_transform(word_vectors)
    
    scaler = StandardScaler()
    scaler.fit(word_vectors)
    return scaler.transform(word_vectors)

In [22]:
word_vectors_pca = get_pca_projection(word_vectors)

assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1 - word_vectors_pca.std(0))) < 1e-5, "points must have unit variance"

Let's visualize the embeddings:

In [23]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    output_notebook()
    
    if isinstance(color, str): 
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: 
        pl.show(fig)
    return fig

In [24]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

Loading BokehJS ...

Figure(id='1003', ...)

### T-SNE

There is a more complicated method of data visualization. It's called t-SNE. You can gain an intuition behind it from [this](https://distill.pub/2016/misread-tsne/) article (warning: even more beautiful illustrations).

**Task** Well, the same as the previous one: apply [TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html), normalize and center the result.

In [25]:
from sklearn.manifold import TSNE

def get_tsne_projection(word_vectors):
    # <fill me>
    tsne = TSNE(n_components=2)
    word_vectors = tsne.fit_transform(word_vectors)
    
    scaler = StandardScaler()
    scaler.fit(word_vectors)
    return scaler.transform(word_vectors)

In [26]:
word_tsne = get_tsne_projection(word_vectors)
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Loading BokehJS ...

Figure(id='1101', ...)

## Using Pretrained Embeddings

We can also use a pretrained embeddings model. There are a number of such models in gensim, you can call `api.info()` to get the list.

Let's load a model:

In [27]:
import gensim.downloader as api

model = api.load('glove-twitter-100')

## Building Phrase Embeddings

The simplest way to obtain a phrase embedding is to average embeddings of the words in the phrase.

*You are probably thinking, 'What a dumb idea, why on earth the average of embedding should contain any useful information'. Well, check [this paper](https://arxiv.org/pdf/1805.09843.pdf).*

Let's do it: tokenize and lowercase the texts, calc the mean embedding for the words with known embeddings.

**Task** Implement the following function.

In [30]:
def get_phrase_embedding(model, phrase):
    """ Calcs phrase embedding as a mean of known word embeddings in the phrase. 
    If all the words are unknown, returns zero vector.
    :param model: KeyedVectors instance
    :param phrase: str or list of str (tokenized text)
    """    
    embedding = np.zeros([model.vector_size], dtype='float32')
    
    if isinstance(phrase, str):
        words = word_tokenize(phrase.lower())
    else:
        words = phrase
    
    # <implement me>
    words = [*filter(lambda word: word in model.key_to_index, words)]
    
    if len(words) == 0:
        embedding = np.zeros([model.vector_size], dtype='float32')
    else:
        embedding = np.mean([*map(model.get_vector, words)], axis = 0, dtype='float32')
    
    return embedding

In [31]:
vector = get_phrase_embedding(model, "I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.30757686, -0.05861897,  0.143751  , -0.11104885, -0.96929336,
                             -0.21928601,  0.21652265,  0.14978765,  1.4842536 ,  0.017826  ],
                              dtype=np.float32))

Well, we are ready to embed all the sentences in our corpus.

In [32]:
text_vectors = np.array([get_phrase_embedding(model, phrase) for phrase in tokenized_texts])

In [33]:
text_vectors.shape

(537361, 100)

What can we do with it? Now we are able perform search of the nearest neighbours to the given phrase in our base!

How are we going to define the distance?

We'll use cosine similarity of two vectors:
$$\text{cosine_similarity}(x, y) = \frac{x^{T} y}{||x||\cdot ||y||}$$

*It's not a [distance](https://www.encyclopediaofmath.org/index.php/Metric) strictly speaking but we still can use it to search for the vectors.*

**Task** Calc the similarity between `query` embedding and `text_vectors` using `cosine_similarity` function. Find `k` vectors with highest scores and return corresponding texts from `texts` list.

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

k=10
query = 'How do i enter the matrix?'
query_vector = get_phrase_embedding(model, query)
similarities = cosine_similarity(text_vectors, [query_vector])
nearest = np.argsort(similarities, axis=None)[-k:]
nearest

array([ 93960, 449642, 181904, 492921, 193900, 171348, 273668, 119429,
        30950, 187447], dtype=int64)

In [37]:
np.take(texts, nearest[::-1])

array(["How do I download the Mengto's Design-Code book?",
       'How do I get to the dark web?',
       'What should I do to enter hollywood?',
       'How do I use the Greenify app?',
       'What can I do to save the world?', 'How do I win this?',
       'How do I think out of the box? How do I learn to think out of the box?',
       'How do I find the 5th dimension?', 'How do I use the pad in MMA?',
       'How do I estimate the competition?'], dtype='<U1169')

In [38]:
def find_nearest(model, text_vectors, texts, query, k=10):
    # <implement me too>
    query_vector = get_phrase_embedding(model, query)
    similarities = cosine_similarity(text_vectors, [query_vector])
    nearest = np.argsort(similarities, axis=None)[-k:]
    return np.take(texts, nearest[::-1])

In [39]:
results = find_nearest(model, text_vectors, texts, query="How do i enter the matrix?", k=10)

print('\n'.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[1] == 'How do I get to the dark web?'
assert results[4] == 'What can I do to save the world?'

How do I download the Mengto's Design-Code book?
How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?


In [40]:
find_nearest(model, text_vectors, texts, query="How does Trump?", k=10)

array(['What does Donald Trump think about Israel?',
       'What books does Donald Trump like?',
       'What does Donald Trump think of India?',
       'What does India think of Donald Trump?',
       'What does Donald Trump think of China?',
       'What does Donald Trump think about Pakistan?',
       'What companies does Donald Trump own?',
       'What does Dushka Zapata think about Donald Trump?',
       'Does Donald Trump have dementia/alzheimers?',
       'How does it feel to date Ivanka Trump?'], dtype='<U1169')

In [41]:
find_nearest(model, text_vectors, texts, query="Why don't i ask a question myself?", k=10)

array(["Why do you always answer a question with a question? I don't, or do I?",
       'Why do I ask this question?', 'How do I ask a question?',
       'How do I ask a question on this?',
       "Why do I have to ask a girl out? Why can't she ask me?",
       'How do I downvote a question?', 'How do I ask someone on a date?',
       'Why do I question myself about this?',
       "How do I ask a girl I don't know to fuck?",
       'How do you ask a question?'], dtype='<U1169')

## Starting Classification

### Bag-of-Words

Finally, we are ready to return to the classification task.

We have two sentences and we are going calculate their similarity and compare it with some threshold. If the value is higher than the threshold then we'll call the sentences similar.

Let's start with tokenization of the questions.

In [42]:
tokenized_question1 = [word_tokenize(question.lower()) for question in quora_data.question1]
tokenized_question2 = [word_tokenize(question.lower()) for question in quora_data.question2]

In [43]:
assert tokenized_question1[0] == ['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india', '?']
assert tokenized_question2[2] == ['how', 'can', 'internet', 'speed', 'be', 'increased', 'by', 'hacking', 'through', 'dns', '?']

**Task** Calc the cosine similarity between the questions.

In [ ]:
question1_vectors = <calc vectors for tokenized_question1>
question2_vectors = <calc vectors for tokenized_question2>
# 1:37 
cosine_similarities = <calc similarities between the vectors in question1_vectors and question2_vectors>

In [ ]:
assert cosine_similarities.shape == (len(quora_data),), 'Check the shapes'

target_similarity = cosine_similarity([get_phrase_embedding(model, tokenized_question1[1])], 
                                      [get_phrase_embedding(model, tokenized_question2[1])])[0, 0]
assert np.allclose(cosine_similarities[1], target_similarity), 'Check your calculations'

Let's find the texts' similarity threshold.

We are going to optimize accuracy of the similarity prediction. For instance, accuracy with threshold equal to 0 would be equal to the fraction ones in the dataset:

In [ ]:
(quora_data.is_duplicate == 1).mean()

**Task** Implement the `accuracy` function that calculates accuracy with the given threshold.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def accuracy(cosine_similarities, threshold, labels):
    return <implement me>

thresholds = np.linspace(0, 1, 100, endpoint=False)
plt.plot(thresholds, [accuracy(cosine_similarities, th, quora_data.is_duplicate) for th in thresholds])

Let's optimize over this function to find the optimal threshold.

In [ ]:
from scipy.optimize import minimize_scalar

res = minimize_scalar(
    lambda th: -accuracy(cosine_similarities, th, quora_data.is_duplicate), bounds=(0.5, 0.99), method='bounded'
)

best_threshold = res.x
best_accuracy = accuracy(cosine_similarities, best_threshold, quora_data.is_duplicate)
print('Threshold = {:.5f}, Accuracy = {:.2%}'.format(best_threshold, best_accuracy))

assert best_accuracy > 0.65, 'Check yourself'

Well, we are a bit better than random :)

### Tf-idf Weights

The averaging of vectors is boring. We can use weighted average - with tf-idf weights.

Let's use `TfidfVectorizer` for this task.

You see, `TfidfVectorizer` returns matrix `(samples_count, words_count)`. Our embeddings is a matrix `(words_count, embedding_dim)`:

In [ ]:
model.vectors.shape

The embedding of a sequence of words $w_1, \ldots, w_k$, as we defined, it is vector $\sum_i \text{idf}(w_i) \cdot \text{embedding}(w_i)$.

That means that we can multiply matrices `(samples_count, words_count) x (words_count, embedding_dim)` to obtain the embeddings for all phrases we have.

But we need to have corresponding words in both matrices. That is i-th row in the first matrix correspond to the i-th column in the second matrix.

To achieve it, we are going to use `vocabulary` argument of `TfidfVectorizer`.

We can extract the vocabulary this way from the gensim model:

In [ ]:
vocabulary = {word: vocab_element.index for word, vocab_element in model.vocab.items()}

Initialize the vectorizer:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(vocabulary=vocabulary)

vectorizer.fit(texts)

**Task** Apply `vectorizer` to the `quora_data` questions and obtain the phrase vectors by multiplying them on `model.vectors`.

In [ ]:
tfidf_question1 = <calc it>
tfidf_question2 = <and it>

In [ ]:
assert tfidf_question1.shape == tfidf_question2.shape == (len(quora_data), len(vocabulary))

Check, that the text in matrices is correctly encoded:

In [ ]:
for col in tfidf_question1[0].tocoo().col:
    print(model.index2word[col], end=' ')

print('\n' + ' '.join(tokenized_question1[0]))

Now we are able to convert the vectors matrices to vectors. That is, multiply tfidf and word2vec matrices and nomalize the result by the number of words in each sentence.

**Task** Build the question vectors.

In [ ]:
EPS = 1e-9

question1_elements_count = <calc it, add EPS to ensure you don't divide by zero>
question2_elements_count = <and it too>

assert question1_elements_count.shape == question2_elements_count.shape == (len(quora_data), 1)
assert np.all(question1_elements_count > 0) and np.all(question2_elements_count > 0.)

question1_vectors = <calc mean tfidf-weighted vectors>
question2_vectors = <and these too>

assert question1_vectors.shape == question2_vectors.shape == (len(quora_data), model.vectors.shape[1])

assert np.allclose(question1_vectors[0][:10], [ 0.04672134, -0.00910798,  0.06817335,  0.00792347,  0.00907249,
                                                0.05163505,  0.02648487, -0.05109346,  0.04752091, -0.01203835])

**Task** Evaluate the quality of these embeddings.

In [ ]:
cosine_similarities = <calc them>
assert cosine_similarities.shape == (len(quora_data),), 'Check the shapes'
assert np.allclose(cosine_similarities[:5], [0.99604267, 0.9558047 , 0.973884  , 0.79243606, 0.92760015])

In [ ]:
res = minimize_scalar(
    lambda th: -accuracy(cosine_similarities, th, quora_data.is_duplicate), bounds=(0.5, 0.99), method='bounded'
)

best_threshold = res.x
best_accuracy = accuracy(cosine_similarities, best_threshold, quora_data.is_duplicate)
print('Threshold = {:.5f}, Accuracy = {:.2%}'.format(best_threshold, best_accuracy))

## Implementing Word-level Machine Translation

In [ ]:
!wget -O ukr_rus.train.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vAK0SWXUqei4zTimMvIhH3ufGPsbnC_O"
!wget -O ukr_rus.test.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1W9R2F8OeKHXruo2sicZ6FgBJUTJc8Us_"
!wget -O fairy_tale.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1sq8zSroFeg_afw-60OmY8RATdu_T1tej"

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '1d7OXuil646jUeDS1JNhP9XWlZogv6rbu'})
downloaded.GetContentFile('cc.ru.300.vec.zip')

downloaded = drive.CreateFile({'id': '1yAqwqgUHtMSfGS99WLGe5unSCyIXfIxi'})
downloaded.GetContentFile('cc.uk.300.vec.zip')

!unzip cc.ru.300.vec.zip
!unzip cc.uk.300.vec.zip

Let's implement a simple machine translator.

The idea is based on the paper [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087.pdf). There are lots of interesting things in the repo: [https://github.com/facebookresearch/MUSE](https://github.com/facebookresearch/MUSE).

And we are going to translate from Ukrainian to Russian. They are quite similar languages with similar syntax. This is why we can substitute words from one language with words from another and expect something coherent in the result.

That is, we are going to learn how embeddings from one language correspond to embeddings from another, like this:

![](https://raw.githubusercontent.com/facebookresearch/MUSE/master/outline_all.png)

Than we will simply map the source word (the word in the sentence we want to translate) to the target embedding space and take the word with the nearest embedding.

In [ ]:
from gensim.models import KeyedVectors

ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

Look at the pair `серпень-август` (which are translation, means august).

In [ ]:
ru_emb.most_similar([ru_emb["август"]])

In [ ]:
uk_emb.most_similar([uk_emb["серпень"]])

In [ ]:
ru_emb.most_similar([uk_emb["серпень"]])

In [ ]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)


uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

### Learning the mapping from the embedding spaces

We have pairs of corresponding words. So we have to find a mapping which would map their embeddings to be as near as possible.

$$W^*= \arg\min_W ||WX - Y||_F, \text{where} ||*||_F - \text{Frobenius norm}$$

This function is similar to the linear regression (without bias).

**Task** Implement it - use `LinearRegression` from sklearn with `fit_intercept=False`:

In [ ]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False).fit(X_train, Y_train)

Check it:

In [ ]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

Expected that the top contains different months, but `август` is not the first.

We are going to evaluate the mapping by precision@k metric with k = 1, 5, 10.

**Task** Implement following function:

In [ ]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    <implement it>
    return precision_val

In [ ]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [ ]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [ ]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

### Improving Mapping

It can be proven that the mapping with orthogonal constraint is better:
$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

You can find it using SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$

$$W^*=UV^T$$

**Task** Implement the function:

In [ ]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """    
    <calculate it>

In [ ]:
W = learn_transform(X_train, Y_train)

In [ ]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

In [ ]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

### Writing the translator

Now we are ready to implement the translation function. It should find the nearest vector in the target (Russian) embedding space and return the source word if it is not in the embeddings.

In [ ]:
with open("fairy_tale.txt", "r") as in f:
    uk_sentences = [line.rstrip().lower() for line in in f]

In [ ]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    <implement it>

In [ ]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [ ]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

# Supplementary Materials

## To read
### Basic knowledge:  
[On word embeddings - Part 1, Sebastian Ruder](http://ruder.io/word-embeddings-1/)  
[Deep Learning, NLP, and Representations, Christopher Olah](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/)  

### How to clusterize embeddings:  
[Making Sense of Word Embeddings (2016), Pelevina et al](http://anthology.aclweb.org/W16-1620)    

### How to evaluate embeddings:
[Evaluation methods for unsupervised word embeddings (2015), T. Schnabel](http://www.aclweb.org/anthology/D15-1036)  
[Intrinsic Evaluation of Word Vectors Fails to Predict Extrinsic Performance (2016), B. Chiu](https://www.aclweb.org/anthology/W/W16/W16-2501.pdf)  
[Problems With Evaluation of Word Embeddings Using Word Similarity Tasks (2016), M. Faruqui](https://arxiv.org/pdf/1605.02276.pdf)  
[Improving Reliability of Word Similarity Evaluation by Redesigning Annotation Task and Performance Measure (2016), Oded Avraham, Yoav Goldberg](https://arxiv.org/pdf/1611.03641.pdf)  
[Evaluating Word Embeddings Using a Representative Suite of Practical Tasks (2016), N. Nayak](https://cs.stanford.edu/~angeli/papers/2016-acl-veceval.pdf)  


## To watch
[Word Vector Representations: word2vec, Lecture 2, cs224n](https://www.youtube.com/watch?v=ERibwqs9p38)

In [ ]:
!pip3 -qq install torch==1.1
!pip -qq install nltk==3.2.5
!pip -qq install gensim==3.6.0
!pip -qq install bokeh==1.0.4

!wget -O quora.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1ERtxpdWOgGQ3HOigqAMHTJjmOE_tWvoF"
!unzip quora.zip

import nltk
nltk.download('punkt')

In [ ]:
import time
import math
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 

from IPython.display import clear_output
%matplotlib inline

np.random.seed(42)

# Introduction to PyTorch

PyTorch is one of the most well-known frameworks for building neural networks (which is what we're gonna do in this course).

The most obvious alternative is Tensorflow, but right now (at fall of 2018) it's much less user-friendly so we'll stick to pytorch.

And come on, if you learn one of them, you'll be able to learn another.

## Automatic Differentiation

Let's start with one of the fundamental pytorch concepts - automatic differentiation (autograd).

### Computational Graphs

Computational graphs provide a very convenient way to represent functions and calculate their gradients.

For instance,
$$f = (x + y) \cdot z$$

Can be represented with this graph:  
![graph](https://github.com/DanAnastasyev/DeepNLP-Course/raw/master/Week%2003/Images/Circuit.png)  
*From [Backpropagation, Intuitions - CS231n](http://cs231n.github.io/optimization-2/)*

*The forward pass* computes value of the function (green numbers). It starts from the inputs (on the left) and applies the sequence of functions.

*The backward pass* (or *back propagation*) is designed to compute gradients of the function. That is $\frac{\partial f}{\partial x}, \frac{\partial f}{\partial y}, \frac{\partial f}{\partial z}$ in our case. It starts from the output and applies *chain rule* to compute them.

For instance, for $f = q \cdot z$, we have $\frac{\partial f}{\partial q} = z$ and $\frac{\partial f}{\partial z} = q$.  
For $q = x + y$, we have $\frac{\partial q}{\partial x} = \frac{\partial q}{\partial y} = 1$.  
Finally, we can apply the chain rule: $\frac{\partial f}{\partial x} = \frac{\partial f}{\partial q} \frac{\partial q}{\partial x} = z$.

*If you had problems with understanding the stuff above (and even if didn't), check this great tutorial: [Backpropagation, Intuitions - CS231n](http://cs231n.github.io/optimization-2/).*

Well, such calculations in pytorch are fairly simple. You just have to describe your function as a sequence of operations, like:

In [ ]:
x = torch.tensor(-2., requires_grad=True)
y = torch.tensor(5., requires_grad=True)
z = torch.tensor(-4., requires_grad=True)

q = x + y
f = q * z

Call it with some arguments and then ask it like, "Hey, calc your grads, please". And the magic happens:

![graph](https://raw.githubusercontent.com/pytorch/pytorch/master/docs/source/_static/img/dynamic_graph.gif)  
*From [github.com/pytorch/pytorch](https://github.com/pytorch/pytorch)*

Pytorch builds graph and performs backward pass - all by itself.

If you already know tensorflow, you'll see the main difference: graph is built dynamically, it hasn't be compiled and stuff. 

Basically, it means that you can debug your code much more easily.

In [ ]:
f.backward()

print('df/dz =', z.grad)
print('df/dx =', x.grad)
print('df/dy =', y.grad)

The call of the `backward()` method calculates gradients for all tensors in graph (except the subgraphs where `requires_grad == False`).

*Read about autograd in pytorch in depth here: [Autograd mechanics](https://pytorch.org/docs/stable/notes/autograd.html).*

Well, the nicest thing about pytorch is that you can use it like you used numpy. You use `ndarray` all the time, right.

There is an analog for it - `tensor`. We just created few of them actually.

It stores data, like `ndarray`:

In [ ]:
x.data

And gradient (unlike `ndarray`):

In [ ]:
x.grad

Add function used to compute the gradient:

In [ ]:
q.grad_fn

And lots of meta-info:

In [ ]:
x.type(), x.shape, x.device, x.layout

Check this tutorial to learn more about tensors: [Deep Learning with PyTorch: A 60 Minute Blitz > What is PyTorch?](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py)

Sometimes we don't want to compute the gradients. To handle this cases (we'll discuss particular examples very soon), you can use context managers ([Locally disabling gradient computation](https://pytorch.org/docs/stable/autograd.html#locally-disabling-gradient-computation)):
```python
torch.autograd.no_grad()
torch.autograd.enable_grad()
torch.autograd.set_grad_enabled(mode)
```

In [ ]:
with torch.autograd.no_grad():
    x = torch.tensor(-2., requires_grad=True)
    y = torch.tensor(5., requires_grad=True)
    q = x + y

z = torch.tensor(-4., requires_grad=True)
f = q * z

f.backward()

print('df/dz =', z.grad)
print('df/dx =', x.grad)
print('df/dy =', y.grad)

Well, the question is why on earth you would want it to use :)

### Warm-up Task

To understand it, let's write a simple linear regression implementation:

In [ ]:
w_orig, b_orig = 2.6, -0.4

X = np.random.rand(100) * 10. - 5.
y_orig = w_orig * X + b_orig

y = y_orig + np.random.randn(100)

plt.plot(X, y, '.')
plt.plot(X, y_orig)
plt.show()

There are two parameters $w$ and $b$. We want them to be as near to $w_{orig}, b_{orig}$ as possible.

What are we going to optimize? Let's optimize MSE loss:
$$J(w, b) = \frac{1}{N} \sum_{i=1}^N || \hat y_i - y_i(w, b)||^2 =\frac{1}{N} \sum_{i=1}^N || \hat y_i - (w \cdot x_i + b)||^2. $$

We can use *gradient descent* algorithm to optimize it (not even stohastic right now):
$$w_{t+1} := w_t - \alpha \cdot \frac{\partial J}{\partial w}(w_t, b_t)$$
$$b_{t+1} := w_t - \alpha \cdot \frac{\partial J}{\partial b}(w_t, b_t)$$

*You see, it would be nice to use backpropagation here.*

**Task** Implement the optimization using pure numpy.

You'll need:
1. Perform the forward pass: $y(w, b) = w \cdot x + b$;
2. Find the gradients $\frac{\partial J}{\partial w}, \frac{\partial J}{\partial b}$ using backward pass;
3. Move $w, b$ in the anti-gradients direction.

In [ ]:
def display_progress(epoch, loss, w, b, X, y, y_pred):
    clear_output(True)
    print('Epoch = {}, Loss = {}, w = {}, b = {}'.format(epoch, loss, w, b))
    plt.plot(X, y, '.')
    plt.plot(X, y_pred)
    plt.show()
    time.sleep(1)


w = np.random.randn()
b = np.random.randn()

alpha = 0.01

for i in range(100):
    <calculate model's output>
    
    <calculate loss>
    
    <calculate gradients>
    
    <update w and b>
    
    if (i + 1) % 5 == 0:
        display_progress(i + 1, loss, w, b, X, y, y_pred)
        
assert np.abs(w - w_orig) < 0.1, 'Something went wrong :('

It's much simpler to implement the same thing using pytorch.

The forward pass will look almost the same. And we've already learnt how to perform the backward pass! Just call `loss.backward()`.

But there are a number of caveats you have to know about. 

First of all, one doen't simply update `w` and `b`. Try this:

In [ ]:
w = torch.randn(1, requires_grad=True)

w -= 1.

It should fail with a message like:  
`RuntimeError: a leaf Variable that requires grad has been used in an in-place operation.`

The issue is in the support of in-place operations in autograd: [In place operations with autograd](https://pytorch.org/docs/stable/notes/autograd.html#in-place-operations-with-autograd).

But actually we are not going to perform an operation that requires gradients. We're just updating the value of the note.

To fight this problem, we can either use `no_grad` context or update the underline data used by the tensor:

In [ ]:
w.data -= 1.

Another thing you should be aware of is that the gradients are accumulating by default. So you have to update them yourself between `loss.backward()` calls:
```python
w.grad.zero_()
b.grad.zero_()
```

**Task** Implement the linear regression on pytorch.

In [ ]:
X = torch.as_tensor(X).float()
y = torch.as_tensor(y).float()

w = torch.randn(1, requires_grad=True)
b = torch.randn(1, requires_grad=True)

for i in range(100):
    <copy forward pass and add backward pass + parameters updates>
    
    if (i + 1) % 5 == 0:
        display_progress(i + 1, loss, w.item(), b.item(), 
                         X.data.numpy(), y.data.numpy(), y_pred.data.numpy())

Much simpler, isn't it? :)

## Word Embeddings (via High-Level PyTorch API)

Let's move now to more high-level API, where all the good neural network parts are implemented. Quite comprehensive description of it is given in this tutorial: [What is torch.nn really?](https://pytorch.org/tutorials/beginner/nn_tutorial.html)

Last time we used gensim to train word2vec model. Now we're ready to implement our own model.

Well, almost ready. We still haven't discussed what word2vec is.

The key idea is simple: you can understand the meaning of the word by his neighbours (the words that appear frequently in its context):  
![contexts](https://image.ibb.co/mnQ2uz/2018_09_17_21_07_08.png)
*From [cs224n, Lecture 2](http://web.stanford.edu/class/cs224n/lectures/lecture2.pdf)*

The first idea is just to use counts of the words in context as a meaningful word vector.

For instance, for such simple corpus:

```
The red fox jumped
The brown fox jumped
```

we'll have following count vectors:
```
        the fox jumped red brown
red   = (1   1    1     0    0)
brown = (1   1    1     0    0)
```

You see, `red` and `brown` have similar vector! The problem is almost solved. But we have to obtain much smaller embedding vectors.

And here is what word2vec algorithms do. They build embedding vectors based on the neighbours of the word in corpus.

A nice introduction is given in this post: [king - man + woman is queen; but why?](http://p.migdal.pl/2017/01/06/king-man-woman-queen-why.html)

Let's do some preparation work before moving to the interesting stuff.

**Task** Tokenize and lower-case texts.

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize

quora_data = pd.read_csv('train.csv')

quora_data.question1 = quora_data.question1.replace(np.nan, '', regex=True)
quora_data.question2 = quora_data.question2.replace(np.nan, '', regex=True)

texts = list(pd.concat([quora_data.question1, quora_data.question2]).unique())

tokenized_texts = [<do it there>]

assert len(tokenized_texts) == len(texts)
assert isinstance(tokenized_texts[0], list)
assert isinstance(tokenized_texts[0][0], str)

Collect the indices of the most frequent words:

In [ ]:
from collections import Counter

MIN_COUNT = 5

words_counter = Counter(token for tokens in tokenized_texts for token in tokens)
word2index = {
    '<unk>': 0
}

for word, count in words_counter.most_common():
    if count < MIN_COUNT:
        break
        
    word2index[word] = len(word2index)
    
index2word = [word for word, _ in sorted(word2index.items(), key=lambda x: x[1])]
    
print('Vocabulary size:', len(word2index))
print('Tokens count:', sum(len(tokens) for tokens in tokenized_texts))
print('Unknown tokens appeared:', sum(1 for tokens in tokenized_texts for token in tokens if token not in word2index))
print('Most freq words:', index2word[1:21])

### Skip-Gram Word2vec

Word2vec is actually a set of models used to build word embeddings.

We are going to start with the *skip-gram model*.

It's a very simple neural network with just two layers. It aims to build word vectors that encode information about the co-occurring words:  
![](https://i.ibb.co/nL0LLD2/Word2vec-Example.jpg)  
*From cs224n, Lecture 2*

More precisely, it models the probabilities $\{P(w_{c+j}|w_c):  j = c-k, ..., c+k, j \neq c\}$, where $k$ is the context window size, $c$ is index of the central word (which embedding we are trying to optimize).

The learnable parameters of the model are following: matrix $U$ (embeddings' matrix that is used in all downstream tasks. In gensim it's called `syn0`) and matrix $V$ - output layer of the model (in gensim it's called `syn1`).

Two vectors correspond to each word: a row in $U$ and a column in $V$. That is $U \in \mathbb{R}^{|V|, d}$ and $V \in \mathbb{R}^{d, |V|}$, where $d$ is embedding size and $|V|$ is the vocabulary size.

As a result, the neural network looks this way:  
![skip-gram](https://i.ibb.co/F54XzDC/SkipGram.png)

What's going on and how it is connected to probability and word context?

Well, the word is mapped to its embedding $u_c$. Then this embedding is multiplied to matrix $V$. 

As a result, we obtain the set of scores $\{v_j^T u_c : j \in {0, \ldots, |V|}\}$. Each corresponds to the similarity between the word $w_j$ vector and our word vector. It's very similar to the cosine similarity we calculated in the previous lesson, but without normalization.

This similarities show how likely $w_j$ can be in context of word $w_c$. That means, that they can be converted to probability using the softmax function:
$$P(w_j | w_c) = \frac{\exp(v_{j}^T u_c)}{\sum_{i=1}^{|V|} \exp(v_i^T u_c)}.$$

So for each word we calculate such probability distribution over our vocabulary. It's shown in using blue bars in the picture above. More likely word - bluer is the corresponding cell.

The model learns to distribute the probabilities between the co-occuring words for the given one. We'll use cross-entropy loss for it:
$$-\sum_{-k \leq j \leq k, j \neq 0} \log \frac{\exp(v_{c+j}^T u_c)}{\sum_{i=1}^{|V|} \exp(v_i^T u_c)} \to \min_{U, V}.$$

For instance, for the sample from the picture model will be penalized if it outputs a low probability of word `over`.

Please, notice that we calculate the similarity between vectors from different vector spaces. $u_c$ is the vector from the input embeddings and $v_j$ is the vector from the output embeddings. A high similarity between them means that they co-occur frequently, not that they are similar in the syntactic role or their semantics.

On the other hand, the similarity between $u_k$ and $u_m$ means that their output distributions are similar. And that means exactly that the similarity of the count vectors we discussed before and also most probably means their syntactic or semantic similarity.

Check this demo to understand what's going on in more depth: [https://ronxin.github.io/wevi/](https://ronxin.github.io/wevi/).

Let's implement it now!

#### Batches Generations

First of all, we need to collect all the contexts from our corpus.

In [ ]:
def build_contexts(tokenized_texts, window_size):
    contexts = []
    for tokens in tokenized_texts:
        for i in range(len(tokens)):
            central_word = tokens[i]
            context = [tokens[i + delta] for delta in range(-window_size, window_size + 1) 
                       if delta != 0 and i + delta >= 0 and i + delta < len(tokens)]

            contexts.append((central_word, context))
            
    return contexts

In [ ]:
contexts = build_contexts(tokenized_texts, window_size=2)

Check, what you got:

In [ ]:
contexts[:5]

Let's convert words to indices:

In [ ]:
contexts = [(word2index.get(central_word, 0), [word2index.get(word, 0) for word in context]) 
            for central_word, context in contexts]

Neural networks are optimized using stochastic gradient descent methods. Which means, we need a batch generator - a function that generates samples to optimize neural network with.

A simple batch generator looks this way:

In [ ]:
import random

def make_skip_gram_batches_iter(contexts, window_size, num_skips, batch_size):
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * window_size
    
    central_words = [word for word, context in contexts if len(context) == 2 * window_size and word != 0]
    contexts = [context for word, context in contexts if len(context) == 2 * window_size and word != 0]
    
    batch_size = int(batch_size / num_skips)
    batches_count = int(math.ceil(len(contexts) / batch_size))
    
    print('Initializing batches generator with {} batches per epoch'.format(batches_count))
    
    while True:
        indices = np.arange(len(contexts))
        np.random.shuffle(indices)

        for i in range(batches_count):
            batch_begin, batch_end = i * batch_size, min((i + 1) * batch_size, len(contexts))
            batch_indices = indices[batch_begin: batch_end]

            batch_data, batch_labels = [], []

            for data_ind in batch_indices:
                central_word, context = central_words[data_ind], contexts[data_ind]
                
                words_to_use = random.sample(context, num_skips)
                batch_data.extend([central_word] * num_skips)
                batch_labels.extend(words_to_use)
            
            yield {
                'tokens': torch.cuda.LongTensor(batch_data), 
                'labels': torch.cuda.LongTensor(batch_labels)
            }

Check it:

In [ ]:
batch = next(make_skip_gram_batches_iter(contexts, window_size=2, num_skips=2, batch_size=32))

batch

#### nn.Sequential

The simplest way to implement a model on pytorch is to use `nn.Sequential`. Just define the order of layers and it will apply them sequentially.

In [ ]:
model = nn.Sequential(
    nn.Embedding(len(word2index), 32),
    nn.Linear(32, len(word2index))
)

Yep, we've just defined our skip-gram model! 

The construction above says that we need a `nn.Embedding` layer (a layer that maps index to a vector. In our case it would be an index from the `range(len(word2index))` and a 32-dimensional vector) following by a `nn.Linear` layer (just a dot-product of an input vector to the learnable matrix with addition of a learnable bias).

There is another pytorch's feature we haven't discussed yet. It's computations on GPU. Neural networks are usually trained on GPUs because it's much faster.

It's extremely easy to ask pytorch to perform calculations on the GPU. Just call:

In [ ]:
model.cuda()

or

In [ ]:
device = torch.device("cuda")

model = model.to(device)

We'll apply it to the data in the batch:

In [ ]:
tokens, labels = batch['tokens'], batch['labels']

Now, to perform the forward pass just call the model with its input:

In [ ]:
logits = model(batch)

We can calculate the loss now:

In [ ]:
criterion = nn.CrossEntropyLoss()

loss = criterion(logits, labels)

And, of course, perform the backward pass:

In [ ]:
loss.backward()

Finally, we have to update the parameters. We can do it as before (like, `w.data -= alpha * w.grad`). But pytorch contains predefined optimizers that can do the same things (and more complicated stuff).

We are going to use Adam optimizer.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001) 

To update the weights just call `optimizer.step()`:

In [ ]:
print(model[1].weight)

optimizer.step()

print(model[1].weight)

And finally, don't forget to zero grads!

In [ ]:
optimizer.zero_grad()

You can ask optimizer to do it for you, you see.

#### Train Cycle

We are ready to implement the training cycle - just like we did for our linear regression.

**Task** Implement the cycle. Train the model for a few epochs.

In [ ]:
loss_every_nsteps = 1000
total_loss = 0
start_time = time.time()

for step, (batch, labels) in enumerate(make_skip_gram_batches_iter(contexts, window_size=2, num_skips=4, batch_size=128)):
    <1. convert data to tensors>
    
    <2. make forward pass>

    <3. make backward pass>

    <4. apply optimizer>
    
    <5. zero grads>

    total_loss += loss.item()
    
    if step != 0 and step % loss_every_nsteps == 0:
        print("Step = {}, Avg Loss = {:.4f}, Time = {:.2f}s".format(step, total_loss / loss_every_nsteps, 
                                                                    time.time() - start_time))
        total_loss = 0
        start_time = time.time()

#### Analysis

To get the embedding matrix, cast the following magic:

In [ ]:
embeddings = model[0].weight.data.cpu().numpy()

That is, we get the weights from the first layer of the model, move them from gpu to cpu and convert to numpy array.

Let's check how adequate are similarities that the model learnt.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def most_similar(embeddings, index2word, word2index, word):
    word_emb = embeddings[word2index[word]]
    
    similarities = cosine_similarity([word_emb], embeddings)[0]
    top10 = np.argsort(similarities)[-10:]
    
    return [index2word[index] for index in reversed(top10)]

most_similar(embeddings, index2word, word2index, 'warm')

And visualizations!

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook

from sklearn.manifold import TSNE
from sklearn.preprocessing import scale


def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    output_notebook()
    
    if isinstance(color, str): 
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: 
        pl.show(fig)
    return fig


def get_tsne_projection(word_vectors):
    tsne = TSNE(n_components=2, verbose=100)
    return scale(tsne.fit_transform(word_vectors))
    
    
def visualize_embeddings(embeddings, index2word, word_count):
    word_vectors = embeddings[1: word_count + 1]
    words = index2word[1: word_count + 1]
    
    word_tsne = get_tsne_projection(word_vectors)
    draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)
    
    
visualize_embeddings(embeddings, index2word, 100)

### Continuous Bag of Words (CBoW)

Here is an alternative word2vec model:

![](https://i.ibb.co/StXTMFH/CBOW.png)

Now we have to predict the word by its context. The context is represented as a sum of context vectors.

**Task** Implement the batch generator. It should output a context matrix `(samples_count, 2 * window_size)` which contains the context word indices and a target matrix `(samples_count)` with the central word indices.

In [ ]:
def make_cbow_batches_iter(contexts, window_size, batch_size):
    data = np.array([context for word, context in contexts if len(context) == 2 * window_size and word != 0])
    labels = np.array([word for word, context in contexts if len(context) == 2 * window_size and word != 0])
        
    batches_count = int(math.ceil(len(data) / batch_size))
    
    print('Initializing batches generator with {} batches per epoch'.format(batches_count))
    
    while True:
        indices = np.arange(len(data))
        np.random.shuffle(indices)

        for i in range(batches_count):
            <implement the generator>

Check it:

In [ ]:
window_size = 2
batch_size = 32

batch = next(make_cbow_batches_iter(contexts, window_size=window_size, batch_size=batch_size))

assert isinstance(batch, dict)
assert 'labels' in batch and 'tokens' in batch

assert isinstance(batch['tokens'], torch.cuda.LongTensor)
assert isinstance(batch['labels'], torch.cuda.LongTensor)

assert batch['tokens'].shape == (batch_size, 2 * window_size)
assert batch['labels'].shape == (batch_size,)

The alternative way to define a model is to inherit it from `nn.Module`. It's a more flexible approach than defining a `nn.Sequential` model so we are going to use it mostly

```python
class MyNetModel(nn.Module):
    def __init__(self, *args, **kwargs):
        super(MyNetModel, self).__init__()
        <initialize layers>
        
    def forward(self, inputs):
        <apply layers>
        return final_output
```

You have to create all the learnable parameters (usually - layers) of the model in the `__init__` method and you have to apply them in the `forward` method.

**Task** Build the `CBoWModel`.

In [ ]:
class CBoWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        
        self._embeddings = nn.Embedding(vocab_size, embedding_dim)
        self._out_layer = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs):
        <apply the layers>

Check it:

In [ ]:
model = CBoWModel(vocab_size=len(word2index), embedding_dim=32).cuda()

outputs = model(batch['tokens'])

assert isinstance(outputs, torch.cuda.FloatTensor)
assert outputs.shape == (batch_size, len(word2index))

**Task** Train the model in the same way as before.

In [ ]:
model = CBoWModel(vocab_size=len(word2index), embedding_dim=32).cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

loss_every_nsteps = 1000
total_loss = 0
start_time = time.time()

for step, batch in enumerate(make_cbow_batches_iter(contexts, window_size=2, batch_size=128)):
    <copy-paste the learning cycle>

    total_loss += loss.item()
    
    if step != 0 and step % loss_every_nsteps == 0:
        print("Step = {}, Avg Loss = {:.4f}, Time = {:.2f}s".format(step, total_loss / loss_every_nsteps, 
                                                                    time.time() - start_time))
        total_loss = 0
        start_time = time.time()

Let's visualize what we got.

In [ ]:
visualize_embeddings(model.embeddings.weight.data.cpu().numpy(), index2word, 1000)

### Negative Sampling

What is the most computationally hard part of the model optimization? It's computation of softmax function over the vocabulary.

To improve the model's performance *negative sampling* can be used.

The idea is fairly obvious: let's predict the probability that the word $w$ can be in the context $c$: $P(D=1|w,c)$.

The probability (again!) would be a function of the similarity between vectors: 
$$P(D=1|w, c) = \sigma(v_w^T u_c) = \frac 1 {1 + \exp(-v^T_w u_c)}.$$

The sigmoid function just maps the similarity to [0, 1] range.

Well, we have positive samples (word-context pairs from the corpus), but we need some negative samples too to train our model. And we can generate them!  
![Negative Sampling](https://i.ibb.co/D7rTdbr/Negative-Sampling.png)

Just sample random word instead of the correct one and hope that they don't fit the context too well.

The loss function (in CBoW setup) will be following:
$$-\log \sigma(v_c^T u_c) - \sum_{k=1}^K \log \sigma(-\tilde v_k^T u_c),$$
where $v_c$ - central word vector, $u_c$ - context vector (sum of vectors from the context), $\tilde v_1, \ldots, \tilde v_K$ - vectors of the negative samples.

Compare it with ordinary CBoW:
$$-v_c^T u_c + \log \sum_{i=1}^{|V|} \exp(v_i^T u_c).$$

You see, it's quite similar, but the sum is over a much smaller number of samples.

The sampling is performed from $U^{3/4}$, where $U$ - unigram distribution (word frequencies).

We've already calculated the word counts (they were obtained when we called `Counter(words)`). 

Just convert them to frequencies and raise them to the power of $\frac 3 4$. Why exactly $\frac 3 4$? It's just a good constant, but the intuition is following:

$$P(\text{is}) = 0.9, \ P(\text{is})^{3/4} = 0.92$$
$$P(\text{Constitution}) = 0.09, \ P(\text{Constitution})^{3/4} = 0.16$$
$$P(\text{bombastic}) = 0.01, \ P(\text{bombastic})^{3/4} = 0.032$$

The probability of high-frequent words stayed the same (approximately), while low-frequent words now are more likely.

Nice description of this algorithm can be found in [cs224n lecture notes](https://github.com/maxim5/cs224n-winter-2017/blob/master/lecture_notes/cs224n-2017-notes1.pdf).

**Task** Implement Negative Sampling.

Define distribution first:

In [ ]:
words_sum_count = sum(words_counter.values())
word_distribution = np.array([(words_counter[word] / words_sum_count) ** (3 / 4) for word in index2word])
word_distribution /= word_distribution.sum()

indices = np.arange(len(word_distribution))

np.random.choice(indices, p=word_distribution, size=(32, 5))

In [ ]:
class NegativeSamplingModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        
        self._embeddings = nn.Embedding(vocab_size, embedding_dim)
        self._out_layer = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, inputs, targets, num_samples):
        '''
        inputs: (batch_size, context_size)
        targets: (batch_size)
        num_samples: int
        Returns loss calculated like in the formula above
        '''
        
        <calc u_c (using self._embedding) & v_c (using self._out_layer)>
        
        <obtain negative sample indices with shape (inputs.shape[0], num_samples) using np.random.choice>
        
        <calculate v_tilde - embeddings of the negative samples (using self._out_layer)>
        
        <calculate logsigmoid outputs (use F.logsigmoid)>

        <return mean loss>

Train and visualize the model.

In [ ]:
model = NegativeSamplingModel(vocab_size=len(word2index), embedding_dim=32).cuda()

optimizer = optim.Adam(model.parameters(), lr=0.01)  

negative_samples = 20
loss_every_nsteps = 1000
total_loss = 0
start_time = time.time()

for step, batch in enumerate(make_cbow_batches_iter(contexts, window_size=2, batch_size=128)):
    <update the learning cycle accordingly>

    total_loss += loss.item()
    
    if step != 0 and step % loss_every_nsteps == 0:
        print("Step = {}, Avg Loss = {:.4f}, Time = {:.2f}s".format(step, total_loss / loss_every_nsteps, 
                                                                    time.time() - start_time))
        total_loss = 0
        start_time = time.time()

In [ ]:
visualize_embeddings(model.embeddings.weight.data.cpu().numpy(), index2word, 1000)

### Structured Word2Vec

In the paper [Two/Too Simple Adaptations of Word2Vec for Syntax Problems (2015), Ling, Wang, et al.](https://www.aclweb.org/anthology/N/N15/N15-1142.pdf) 
two ways of improvement of the embeddings are discussed: *Structured Skip-gram Model* and *Continuous Window Model*:   
![](https://i.ibb.co/56w8MC8/Structured-Word2vec.png)  
*From Two/Too Simple Adaptations of Word2Vec for Syntax Problems*

In contract to the classic word2vec, each context word has its own embedding matrix. The idea is that the words order is meaningful and by learning the order embeddings learn syntax better.

The disadvantage of such approach is that you have to learn much more parameters in the model.

**Task** Read the paper and implement at least one of the described methods.

In [ ]:
class CWindowModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, window_size):
        super().__init__()
        
        <create layers>

    def forward(self, inputs):
        <apply 'em>

In [ ]:
model = CWindowModel(vocab_size=len(word2index), embedding_dim=32, window_size=2).cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01)  

loss_every_nsteps = 1000
total_loss = 0
start_time = time.time()

for step, batch in enumerate(make_cbow_batches_iter(contexts, window_size=2, batch_size=128)):
    <copy-paste the cycle yet again>

    total_loss += loss.item()
    
    if step != 0 and step % loss_every_nsteps == 0:
        print("Step = {}, Avg Loss = {:.4f}, Time = {:.2f}s".format(step, total_loss / loss_every_nsteps, 
                                                                    time.time() - start_time))
        total_loss = 0
        start_time = time.time()

# Supplementary Materials

## To read
### Blogs
[On word embeddings - Part 1, Sebastian Ruder](http://ruder.io/word-embeddings-1/)  
[On word embeddings - Part 2: Approximating the Softmax, Sebastian Ruder](http://ruder.io/word-embeddings-softmax/index.html)  
[Word2Vec Tutorial - The Skip-Gram Model, Chris McCormick](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)  
[Word2Vec Tutorial Part 2 - Negative Sampling, Chris McCormick](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/) 

### Papers
[Word2vec Parameter Learning Explained (2014), Xin Rong](https://arxiv.org/abs/1411.2738)  
[Neural word embedding as implicit matrix factorization (2014), Levy, Omer, and Yoav Goldberg](http://u.cs.biu.ac.il/~nlp/wp-content/uploads/Neural-Word-Embeddings-as-Implicit-Matrix-Factorization-NIPS-2014.pdf)  

### Enhancing Embeddings
[Two/Too Simple Adaptations of Word2Vec for Syntax Problems (2015), Ling, Wang, et al.](https://www.aclweb.org/anthology/N/N15/N15-1142.pdf)  
[Not All Neural Embeddings are Born Equal (2014)](https://arxiv.org/pdf/1410.0718.pdf)  
[Retrofitting Word Vectors to Semantic Lexicons (2014), M. Faruqui, et al.](https://arxiv.org/pdf/1411.4166.pdf)  
[All-but-the-top: Simple and Effective Postprocessing for Word Representations (2017), Mu, et al.](https://arxiv.org/pdf/1702.01417.pdf)  

### Sentence Embeddings
[Skip-Thought Vectors (2015), Kiros, et al.](https://arxiv.org/pdf/1506.06726)  

### Backpropagation
[Backpropagation, Intuitions, cs231n + next parts in the Module 1](http://cs231n.github.io/optimization-2/)   
[Calculus on Computational Graphs: Backpropagation, Christopher Olah](http://colah.github.io/posts/2015-08-Backprop/)

## To watch
[cs224n "Lecture 2 - Word Vector Representations: word2vec"](https://www.youtube.com/watch?v=ERibwqs9p38&index=2&list=PLqdrfNEc5QnuV9RwUAhoJcoQvu4Q46Lja&t=0s)  
[cs224n "Lecture 5 - Backpropagation"](https://www.youtube.com/watch?v=isPiE-DBagM&index=5&list=PLqdrfNEc5QnuV9RwUAhoJcoQvu4Q46Lja&t=0s)   
